## Dbt 실행 후, seed_dev 및 gold_dev 스키마의 테이블 조회
커널 선택: http://4.230.26.44:8888

### Spark session 생성

In [1]:
import os
import sys

sys.path.append("/app")
from src.utils.spark_builder import get_spark_session
from pyspark.sql import functions as F

os.environ[
    "PYSPARK_SUBMIT_ARGS"
] = """
    --conf spark.cores.max=2
    --conf spark.executor.memory=8g
    --conf spark.app.name=GDELT_dbt_test
    pyspark-shell
"""

# Spark 세션 생성
spark = get_spark_session("GDELT_dbt_test", "spark://spark-master:7077")
print("✅ Spark 세션 생성 완료")

25/09/27 11:04:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/27 11:04:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/09/27 11:04:54 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


✅ Spark 세션 생성 완료


In [2]:
# Spark Metastore에 존재하는 모든 데이터베이스(스키마) 목록을 확인합니다.
print("📖 사용 가능한 데이터베이스(스키마) 목록:")
spark.sql("SHOW DATABASES").show()

📖 사용 가능한 데이터베이스(스키마) 목록:
+------------+
|   namespace|
+------------+
|     default|
|    gold_dev|
|   gold_prod|
|    seed_dev|
|   seed_prod|
|      silver|
| staging_dev|
|staging_prod|
+------------+



In [22]:
# dbt seed 또는 dbt run으로 생성된 테이블 확인
print("📜 '_dev' 스키마의 테이블 목록:")
spark.sql("SHOW TABLES IN seed_dev").show()
spark.sql("SHOW TABLES IN staging_dev").show()
spark.sql("SHOW TABLES IN gold_dev").show()

📜 '_dev' 스키마의 테이블 목록:


+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
| seed_dev|actor_ethnic_grou...|      false|
| seed_dev|actor_organizatio...|      false|
| seed_dev|actor_religion_codes|      false|
| seed_dev|    actor_role_codes|      false|
| seed_dev|  event_detail_codes|      false|
| seed_dev|event_quad_class_...|      false|
| seed_dev|    event_root_codes|      false|
| seed_dev|       geo_adm_codes|      false|
| seed_dev|   geo_country_codes|      false|
| seed_dev|      geo_type_codes|      false|
+---------+--------------------+-----------+



+-----------+--------------------+-----------+
|  namespace|           tableName|isTemporary|
+-----------+--------------------+-----------+
|staging_dev|stg_actors_descri...|      false|
|staging_dev|stg_gkg_detailed_...|      false|
|staging_dev|    stg_seed_mapping|      false|
+-----------+--------------------+-----------+

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
| gold_dev|gold_chart_events...|      false|
| gold_dev|gold_chart_events...|      false|
| gold_dev|gold_chart_weekda...|      false|
| gold_dev|gold_daily_actor_...|      false|
| gold_dev|gold_daily_detail...|      false|
| gold_dev|gold_near_realtim...|      false|
+---------+--------------------+-----------+



### Gold_dev의 모든 테이블 크기 확인용

In [2]:
schema_name = "gold_dev"

try:
    tables_df = spark.sql(f"SHOW TABLES IN {schema_name}")
    table_names = [row.tableName for row in tables_df.collect()]

    if not table_names:
        print(f"❌ '{schema_name}' 스키마에서 테이블을 찾을 수 없습니다.")
    else:
        print(
            f"🥇 '{schema_name}' 스키마에서 총 {len(table_names)}개의 테이블을 발견했습니다.\n"
        )

    for table in table_names:
        full_table_name = f"{schema_name}.{table}"
        print(f"테이블명: {full_table_name}")

        try:
            df = spark.table(full_table_name)
            row_count = df.count()
            col_count = len(df.columns)
            print(f" - 행(Row)의 수: {row_count} 개")
            print(f" - 열(Column)의 수: {col_count} 개\n")

        except Exception as e:
            print(f"❌ 테이블 '{full_table_name}' 조회 중 오류 발생: {e}\n")

except Exception as e:
    print(f"❌ 스키마 '{schema_name}'의 테이블 목록을 가져오는 중 오류 발생: {e}")

🥇 'gold_dev' 스키마에서 총 6개의 테이블을 발견했습니다.

테이블명: gold_dev.gold_chart_events_category


25/09/27 11:05:04 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
25/09/27 11:05:22 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/09/27 11:05:37 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/09/27 11:05:52 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/09/27 11:06:07 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/09/27 11:06:22 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and hav

 - 행(Row)의 수: 13290 개
 - 열(Column)의 수: 8 개

테이블명: gold_dev.gold_chart_events_count_avgtone


 - 행(Row)의 수: 1901 개
 - 열(Column)의 수: 7 개

테이블명: gold_dev.gold_chart_weekday_event_ratio


 - 행(Row)의 수: 1895 개
 - 열(Column)의 수: 7 개

테이블명: gold_dev.gold_daily_actor_network


 - 행(Row)의 수: 1613 개
 - 열(Column)의 수: 17 개

테이블명: gold_dev.gold_daily_detailed_events


 - 행(Row)의 수: 388357 개
 - 열(Column)의 수: 81 개

테이블명: gold_dev.gold_near_realtime_summary


 - 행(Row)의 수: 1901 개
 - 열(Column)의 수: 16 개



### Seed 확인 및 테이블 관리

In [5]:
# Seed 테이블(총 10개)의 샘플 데이터를 5개씩 출력하여 내용 확인
seed_tables = [
    "actor_ethnic_group_codes",
    "actor_organization_codes",
    "actor_religion_codes",
    "actor_role_codes",
    "event_root_codes",
    "event_detail_codes",
    "event_quad_class_codes",
    "geo_adm_codes",
    "geo_country_codes",
    "geo_type_codes",
]

for table in seed_tables:
    print(f"\n 🔍 {table} 테이블 샘플 데이터")
    try:
        spark.sql(f"SELECT * FROM seed_dev.{table} LIMIT 5").show(truncate=False)
    except Exception as e:
        print(f"❌ 테이블 조회 중, 오류 발생: {e}")


 🔍 actor_ethnic_group_codes 테이블 샘플 데이터


25/09/26 16:58:47 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


+----+---------------------+
|code|description          |
+----+---------------------+
|aar |아파르족             |
|abk |압하스인             |
|abr |오스트레일리아 원주민|
|ace |아체인               |
|acg |아창족               |
+----+---------------------+


 🔍 actor_organization_codes 테이블 샘플 데이터
+----+----+--------------------------+
|code|type|description               |
+----+----+--------------------------+
|BHF |정부|보스니아 헤르체고비나 연방|
|FTA |정부|파타                      |
|HAM |정부|하마스                    |
|RPF |정부|르완다 애국 전선          |
|SRP |정부|스릅스카 공화국           |
+----+----+--------------------------+


 🔍 actor_religion_codes 테이블 샘플 데이터
+----+----+---------------+
|code|type|description    |
+----+----+---------------+
|REL |일반|종교 (미지정)  |
|ATH |이념|불가지론/무신론|
|BAH |종교|바하이 신앙    |
|BUD |종교|불교           |
|MAH |종파|대승불교       |
+----+----+---------------+


 🔍 actor_role_codes 테이블 샘플 데이터
+----+--------+--------------+
|code|type    |description   |
+----+--------+--------------+
|COP |1차 역할|경찰          |
|GOV

생성된 테이블을 삭제할 경우, 아래 코드에 해당 스키마와 "삭제할 테이블명"을 수정하여 실행

In [ ]:
tables_to_drop = [
    # "[삭제할 테이블명]"
]
print("🗑️ 기존 테이블 삭제를 시작합니다.")

for table_name in tables_to_drop:
    try:
        spark.sql(f"DROP TABLE IF EXISTS gold_dev.{table_name}")
        print(f"  - 테이블 'gold_dev.{table_name}' 삭제 완료.")
    except Exception as e:
        print(f"  - 테이블 'gold_dev.{table_name}' 삭제 중 오류 발생: {e}")

print("\n✅ 테이블 삭제 작업이 완료되었습니다.")

# 삭제 후 테이블 목록을 다시 확인하여 깨끗해졌는지 확인
print("📜 현재 'gold_dev' 스키마의 테이블 목록:")
spark.sql("SHOW TABLES IN gold_dev").show()

### Staging 폴더의 SQL 파일을 dbt run한 결과 확인

In [7]:
print("🥈 [Staging] stg_seed_mapping 테이블 조회 결과")
spark.sql(
    "SELECT * FROM staging_dev.stg_seed_mapping ORDER BY event_date DESC LIMIT 20"
).show(truncate=False)

🥈 [Staging] stg_seed_mapping 테이블 조회 결과


+---------------+----------+-------------+-------------------+------------------------+----------+-------------+---------------+------------+-----------+------------+-----------------+-----------+----------------------+--------------------------+----------------------------+--------------------------+----------------------+---------------------------+----------------+------------------+-----------------------+--------------+-------------------+-------------------+-----------------------------------------------+-------------------------+--------------------------------------------------------+-------------------------+--------------+---------------+-----------+---------------+--------------------------+----------------------------+--------------------------+----------------------+---------------------------+----------------+------------------+-----------------------+--------------+-------------------+------------------+------------------------------------------------+-------------------

In [8]:
print("🥈 [Staging] stg_actors_description 테이블 조회 결과")
spark.sql(
    "SELECT * FROM staging_dev.stg_actors_description WHERE actor_full_description IS NULL"
).show(truncate=False)

🥈 [Staging] stg_actors_description 테이블 조회 결과


+----------+----------------------+
|actor_code|actor_full_description|
+----------+----------------------+
+----------+----------------------+



In [3]:
spark.sql(
    "SELECT * FROM silver.gdelt_events_detailed WHERE v2_persons IS NOT NULL ORDER BY event_date DESC LIMIT 20"
).show(truncate=False)

+---------------+----------+-----------+--------------+-------------------+-----------------------+------------------+---------------------+---------------------+-----------------+-----------------+-----------------+-----------+-------------------+-------------------+-----------------------+------------------+---------------------+---------------------+-----------------+-----------------+-----------------+-------------+----------+---------------+---------------+----------+---------------+------------+-----------+------------+-----------------+---------------+-------------------------------------------------+-----------------------+--------------------+--------------------+--------------+---------------+---------------------+-------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+-------------------+------------+-------------------

In [ ]:
print("🥈 [Staging] stg_gkg_detailed_events 테이블 조회 결과")
spark.sql(
    "SELECT * FROM staging_dev.stg_gkg_detailed_events WHERE v2_persons IS NOT NULL ORDER BY event_date DESC LIMIT 20"
).show(truncate=False)

### Marts 폴더의 SQL 파일을 dbt run한 결과 확인

In [3]:
print("🥇 [Marts] gold_near_realtime_summary 테이블 조회 결과")
spark.sql(
    "SELECT * FROM gold_dev.gold_near_realtime_summary ORDER BY event_date ASC"
).show(truncate=False)

🥇 [Marts] gold_near_realtime_summary 테이블 조회 결과


+----------+-------------------------+-------------------------------------+-------------------------+-----------+-------------------+--------------------+--------------+-------------+--------------+-----------------------+--------------------+-------------------+-------------------+-------------------+-----------------------------------------------------------------------------------------+
|event_date|mp_action_geo_country_iso|mp_action_geo_country_eng            |mp_action_geo_country_kor|event_count|avg_goldstein_scale|avg_tone            |total_mentions|total_sources|total_articles|count_cooperation_event|count_conflict_event|count_anomaly_event|risk_score_daily   |processed_at       |daily_tone_summary                                                                       |
+----------+-------------------------+-------------------------------------+-------------------------+-----------+-------------------+--------------------+--------------+-------------+--------------+-----------

In [ ]:
print("🥇 [Marts] gold_daily_detailed_events 테이블 조회 결과")
spark.sql(
    "SELECT * FROM gold_dev.gold_daily_detailed_events WHERE amounts IS NOT NULL"
).show(truncate=False)

# spark.sql(
#     "SELECT actor1_info, actor2_info, simple_story, rich_story, headline_story, event_summary, tone_story FROM gold_dev.gold_daily_detailed_events WHERE actor1_info IS NULL"
# ).show(truncate=False)

🥇 [Marts] gold_daily_detailed_events 테이블 조회 결과


25/09/27 11:17:21 WARN DeltaLog: Change in the table id detected while updating snapshot. 
Previous snapshot = Snapshot(path=s3a://warehouse/gold_dev/gold_daily_detailed_events/_delta_log, version=3, metadata=Metadata(05d9c3e5-d77f-4017-b713-9ef5e57d3976,null,null,Format(parquet,Map()),{"type":"struct","fields":[{"name":"global_event_id","type":"string","nullable":true,"metadata":{}},{"name":"event_date","type":"date","nullable":true,"metadata":{}},{"name":"is_root_event","type":"boolean","nullable":true,"metadata":{}},{"name":"mp_event_categories","type":"string","nullable":true,"metadata":{}},{"name":"mp_event_info","type":"string","nullable":true,"metadata":{}},{"name":"quad_class","type":"integer","nullable":true,"metadata":{}},{"name":"mp_quad_class","type":"string","nullable":true,"metadata":{}},{"name":"goldstein_scale","type":"double","nullable":true,"metadata":{}},{"name":"num_mentions","type":"integer","nullable":true,"metadata":{}},{"name":"num_sources","type":"integer","nul

+---------------+----------+-------------+-------------------+--------------+----------+-------------+---------------+------------+-----------+------------+-----------------+-----------+--------------------+--------------------------+--------------------------+--------------------------+----------------------+---------------------------+----------------+------------------+-----------------------+--------------+-------------------+------------------+---------------------------------+-------------------------+----------------------------+-------------------------+--------------+---------------+-----------+--------------------+--------------------------+--------------------------------------------------------+--------------------------+----------------------+---------------------------+----------------+------------------+-----------------------+--------------+-------------------+------------------+-------------------------------+-----------------------+-------------------------+----------

25/09/27 12:18:05 ERROR TaskSchedulerImpl: Lost executor 0 on 10.0.1.58: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
25/09/27 12:18:05 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_188_21 !
25/09/27 12:18:05 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_188_46 !
25/09/27 12:18:05 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_140_48 !
25/09/27 12:18:05 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_90_41 !
25/09/27 12:18:05 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_136_26 !
25/09/27 12:18:05 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_188_38 !
25/09/27 12:18:05 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_38_41 !
25/09/27 12:18:05 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_42_6 !
25/09/27 12:18:05 WARN BlockManager

In [12]:
print("🥇 [Marts] gold_daily_actor_network 테이블 조회 결과")
spark.sql(
    "SELECT * FROM gold_dev.gold_daily_actor_network ORDER BY event_date ASC"
).show(truncate=False)

🥇 [Marts] gold_daily_actor_network 테이블 조회 결과


+----------+------------------+--------------------------------------------------------+----------------------+--------------+---------------+------------------+--------------------------------------------------------+------------------+--------------+---------------+-----------+-----------------+-------------------+---------------------------------------------+----------+-------------+
|event_date|actor1_country_iso|actor1_country_eng                                      |actor1_country_kor    |actor1_geo_lat|actor1_geo_long|actor2_country_iso|actor2_country_eng                                      |actor2_country_kor|actor2_geo_lat|actor2_geo_long|event_count|avg_tone         |processed_at       |location_text                                |route_name|tone_category|
+----------+------------------+--------------------------------------------------------+----------------------+--------------+---------------+------------------+--------------------------------------------------------+--

In [13]:
print("🥇 [Marts] gold_chart_events_category 테이블 조회 결과")
spark.sql(
    "SELECT * FROM gold_dev.gold_chart_events_category ORDER BY event_date ASC"
).show(truncate=False)

🥇 [Marts] gold_chart_events_category 테이블 조회 결과
+----------+-------------------------+--------------------------------------------------------+-------------------------+-------------+-------------------+-----------+-------------------+
|event_date|mp_action_geo_country_iso|mp_action_geo_country_eng                               |mp_action_geo_country_kor|mp_quad_class|mp_event_categories|event_count|processed_at       |
+----------+-------------------------+--------------------------------------------------------+-------------------------+-------------+-------------------+-----------+-------------------+
|2024-09-23|GBR                      |The United Kingdom of Great Britain and Northern Ireland|영국                     |언어적 협력  |협의               |9          |2025-09-23 18:30:00|
|2024-09-23|CAN                      |Canada                                                  |캐나다                   |언어적 협력  |협의               |10         |2025-09-23 20:00:00|
|2024-09-23|USA                 

In [14]:
print("🥇 [Marts] gold_chart_events_count_avgtone 테이블 조회 결과")
spark.sql(
    "SELECT * FROM gold_dev.gold_chart_events_count_avgtone ORDER BY event_date ASC"
).show(truncate=False)

🥇 [Marts] gold_chart_events_count_avgtone 테이블 조회 결과


+----------+-------------------------+--------------------------------------------------------+-------------------------+-----------+---------------------+-------------------+
|event_date|mp_action_geo_country_iso|mp_action_geo_country_eng                               |mp_action_geo_country_kor|event_count|avg_tone             |processed_at       |
+----------+-------------------------+--------------------------------------------------------+-------------------------+-----------+---------------------+-------------------+
|2024-09-23|CAN                      |Canada                                                  |캐나다                   |18         |0.1445921030372805   |2025-09-23 22:45:00|
|2024-09-23|PNG                      |Papua New Guinea                                        |파푸아뉴기니             |3          |1.9228367306155096   |2025-09-23 23:30:00|
|2024-09-23|GMB                      |The Gambia                                              |감비아                   |4          

In [15]:
print("🥇 [Marts] gold_chart_weekday_event_ratio 테이블 조회 결과")
spark.sql(
    "SELECT * FROM gold_dev.gold_chart_weekday_event_ratio ORDER BY period ASC"
).show(truncate=False)

🥇 [Marts] gold_chart_weekday_event_ratio 테이블 조회 결과
+------------------+-------------------------+-------------------------+-------------------------+-----------+------------+------------------+
|period            |mp_action_geo_country_iso|mp_action_geo_country_eng|mp_action_geo_country_kor|weekday_num|weekday_name|event_percentage  |
+------------------+-------------------------+-------------------------+-------------------------+-----------+------------+------------------+
|이번 달 (25년 9월)|ABW                      |Aruba                    |아루바                   |5          |Thu         |17.39130434782609 |
|이번 달 (25년 9월)|ABW                      |Aruba                    |아루바                   |4          |Wed         |39.13043478260870 |
|이번 달 (25년 9월)|ABW                      |Aruba                    |아루바                   |3          |Tue         |43.47826086956522 |
|이번 달 (25년 9월)|AFG                      |Afghanistan              |아프가니스탄             |6          |Fri         |11.

### Spark 세션 종료

In [ ]:
spark.stop()
print("\n✅ Spark 세션이 종료되었습니다.")


✅ Spark 세션이 종료되었습니다.


25/09/27 07:25:53 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/spark-b449e5b0-696d-4883-b655-c030fb8fc2d6/pyspark-2ed5d8a5-7a2d-465c-a004-5f54425f321a. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/spark-b449e5b0-696d-4883-b655-c030fb8fc2d6/pyspark-2ed5d8a5-7a2d-465c-a004-5f54425f321a
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:178)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:113)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:94)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1231)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4(ShutdownHookManager.scala:65)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4$adapted(ShutdownHookManager.scala:62)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreac